##CANet

In [1]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Multiply
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [2]:
def convolutional_block(X, f, filters, stage, block, s = 1):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [3]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [8]:
X_input = Input(shape=(512,512,3))

# Stage 1
X = Conv2D(64, (3, 3), name='conv1', padding="same", kernel_initializer=glorot_uniform(seed=0))(X_input)
X = BatchNormalization(axis=3, name='bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((2, 2), strides=(2, 2))(X)

# Stage 2
X_ag_cat = convolutional_block(X, f=3, filters=[4, 4, 8], stage=2, block='a', s=2)
X = identity_block(X_ag_cat, 3, [4, 4, 8], stage=2, block='b')

# Stage 3 (≈4 lines)
X = convolutional_block(X, f = 3, filters = [8, 8, 16], stage = 3, block='a', s = 2)
X = identity_block(X, 3, [8, 8, 16], stage=3, block='b')
X = identity_block(X, 3, [8, 8, 16], stage=3, block='c')

    # Stage 4 (≈6 lines)
X = convolutional_block(X, f = 3, filters = [16, 16, 32], stage = 4, block='a', s = 1)
X = identity_block(X, 3, [16, 16, 32], stage=4, block='b')
X = identity_block(X, 3, [16, 16, 32], stage=4, block='c')
X = identity_block(X, 3, [16, 16, 32], stage=4, block='d')



    # Stage 5 (≈3 lines)
X = convolutional_block(X, f = 3, filters = [32, 32, 64], stage = 5, block='a', s = 1)
X = identity_block(X, 3, [32, 32, 64], stage=5, block='b')
X = identity_block(X, 3, [32, 32, 64], stage=5, block='c')
X = identity_block(X, 3, [32, 32, 64], stage=5, block='d')
X = identity_block(X, 3, [32, 32, 64], stage=5, block='e')

x_GF = tf.keras.layers.GlobalAveragePooling2D()(X)   
x_GF = tf.expand_dims(x_GF, axis=1)
x_GF = tf.expand_dims(x_GF, axis=1)                    #Global Average pooling changes the dimensions of the output. Instead use average   
x_GF2 = BatchNormalization()(x_GF)                               # pooling with the kernal dimensions equal to that of the entire layer
x_GF3 = Activation('relu')(x_GF2)    
GF_Conv_1 = Conv2D(32, (1, 1), activation = 'relu', padding = 'same', name = 'GF_conv1' )(x_GF3)
GF_Conv_T = (Conv2DTranspose(32 ,(64,64), use_bias = False))(GF_Conv_1)


c1 = Concatenate()([X,GF_Conv_T])
CF_x1 = AveragePooling2D((2, 2),  (2, 2), name = 'CF1_pool1' )(c1)
CF_Conv_11 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same', name = 'CF1_conv1' )(CF_x1)
CF_Conv_12 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same', name = 'CF1_conv2')(CF_Conv_11)
CF_Conv_13 = Conv2D(32, (1, 1), activation = 'relu', padding = 'same', name = 'CF1_conv3' )(CF_Conv_12)
CF_x2 = Activation('relu')(CF_Conv_13)
CF_Conv_14 = Conv2D(32, (1, 1), activation = 'relu', padding = 'same', name = 'CF1-conv4' )(CF_x2)
CF_x3 = Activation('sigmoid')(CF_Conv_14)
CF_x4 = Multiply()([CF_Conv_12 , CF_x3])
CF_x5 = Add()([CF_Conv_12,CF_x4])
CF_Conv_T1 = UpSampling2D( interpolation='bilinear')(CF_x5) 

c12= concatenate([X,CF_Conv_T1])
x = AveragePooling2D((2, 2), (2, 2), name = 'CF2_pool2' )(c12)
CF_Conv_21 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same', name = 'CF2_conv12' )(x)
CF_Conv_22 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same', name = 'CF2_conv22' )(CF_Conv_11)
CF_Conv_23 = Conv2D(32, (1, 1), activation = 'relu', padding = 'same', name = 'CF2_conv33')(CF_Conv_12)
x = Activation('relu')(CF_Conv_13)
CF_Conv_24 = Conv2D(32, (1, 1), activation = 'relu', padding = 'same', name = 'CF2_conv44' )(x)
x =  Activation('sigmoid')(CF_Conv_24)
x = Multiply()([CF_Conv_22 , x])
x = Add()([CF_Conv_22,x])
CF_Conv_T2 = UpSampling2D( interpolation='bilinear')(x)

c3 = concatenate([X,CF_Conv_T2])
x = AveragePooling2D((2, 2),  (2, 2), name = 'CF3_pool3')(c1)
CF_Conv_31 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same', name = 'CF3_conv13')(x)
CF_Conv_32 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same', name = 'CF3_conv23')(CF_Conv_11)
CF_Conv_33 = Conv2D(32, (1, 1), activation = 'relu', padding = 'same', name = 'CF3_conv33' )(CF_Conv_12)
x = Activation('relu')(CF_Conv_13)
CF_Conv_34 = Conv2D(32, (1, 1), activation = 'relu', padding = 'same', name = 'CF3_conv44' )(x)
x = Activation('sigmoid')(CF_Conv_34)
x =  Multiply()([CF_Conv_32 , x])
x = Add()([CF_Conv_32,x])
CF_Conv_T3 = UpSampling2D( interpolation='bilinear')(x)

o = Add(name = 'add1')([GF_Conv_T, CF_Conv_T1, CF_Conv_T2, CF_Conv_T3])

FS_Conv_1 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same', name = 'FS_conv1')(o)
x_GF = tf.keras.layers.GlobalAveragePooling2D()(FS_Conv_1)   
x_GF = tf.expand_dims(x_GF, axis=1)
x_GF = tf.expand_dims(x_GF, axis=1)
FS_Conv_2 = Conv2D(32, (1, 1), activation = 'relu', padding = 'same', name = 'FS_conv2')(x_GF)
bn = BatchNormalization()(FS_Conv_2)
s = Activation('sigmoid')(bn)
x = Multiply()([FS_Conv_1,s])
x = UpSampling2D( interpolation='bilinear')(x)

mid_Conv = Conv2D(32, (3, 3), activation = 'relu', padding = 'same', name = 'FS_Conv3')(x)


AG_Conv_L1 = Conv2D(32, (7, 1), activation = 'relu', padding = 'same', name = 'AG_conv1' )(X_ag_cat)
AG_Conv_L2 = Conv2D(32, (1, 7), activation = 'relu', padding = 'same', name = 'AG_conv2')(AG_Conv_L1)

AG_Conv_R1 = Conv2D(32, (1, 7), activation = 'relu', padding = 'same', name = 'AG_conv3' )(X_ag_cat)
AG_Conv_R2 = Conv2D(32, (7, 1), activation = 'relu', padding = 'same', name = 'AG_conv4')(AG_Conv_R1)

o2 = Add()([AG_Conv_L2,AG_Conv_R2])

AG_Conv_F1 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same', name = 'AG_conv5' )(o2)

o3 = Add()([o2,AG_Conv_F1])
CF = concatenate([mid_Conv,o3])

Final = Conv2D(21, (3, 3), activation = 'relu', padding = 'same', name = 'FinalBlock1')(CF)
    
o_Final  = UpSampling2D((4,4), interpolation = 'bilinear')(Final)
Final = (Activation('softmax'))(o_Final)

In [9]:
model = Model(inputs = X_input, outputs = Final)

model.summary()

Model: "functional_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 512, 512, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 512, 512, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_104 (Activation)     (None, 512, 512, 64) 0           bn_conv1[0][0]                   
_______________________________________________________________________________________